## Création du filtre de liquidité

- idée : filtre dynamique dans le temps
- À chaque date de rebalancement de référence (dernier jour de trading le janvier, avril, juillet et octobre) on calcule la liquidité moyenne des 3 derniers mois à partir des poids de référence de l'indice et des données de volumes.

In [1]:
import matplotlib.pyplot as plt
from base_strategy import *
from preprocessing import read_data, read_gics_sectors, read_reference_index_holdings

In [2]:
df_px = read_data("Constituents PX_LAST data")
df_volume = read_data("Constituents PX_VOLUME data")
df_total_ret = read_data("Constituents TOT_RET_INDEX data")
df_gics = read_gics_sectors()
df_ref = read_reference_index_holdings()

In [3]:
df_volume.head()

,AAC/B CN Equity,AAH CN Equity,AAR-U CN Equity,AAV CN Equity,ABX CN Equity,AC CN Equity,ACB CN Equity,ACE/A CN Equity,ACE/B CN Equity,ACM/A CN Equity,...,WTE CN Equity,WTN CN Equity,WTO CN Equity,X CN Equity,XG CN Equity,YLO CN Equity,YRI CN Equity,ZL CN Equity,ZZZ CN Equity,SPTSX Index
Index Date,,,,,,,,,,,,,,,,,,,,,
2000-01-04,8750.0,9711.0,NaN,16575.0,926579.0,NaN,NaN,NaN,NaN,21396.0,...,25607.0,NaN,NaN,NaN,NaN,NaN,1804.0,510922.0,NaN,62662940.0
2000-01-05,23285.0,11400.0,NaN,31275.0,1452213.0,NaN,NaN,NaN,NaN,6000.0,...,55081.0,NaN,NaN,NaN,NaN,NaN,3357.0,343030.0,NaN,94698130.0
2000-01-06,29305.0,8783.0,NaN,5150.0,600273.0,NaN,NaN,NaN,NaN,38400.0,...,23580.0,NaN,NaN,NaN,NaN,NaN,2107.0,518189.0,NaN,84015360.0
2000-01-07,13695.0,11620.0,NaN,4555.0,1020813.0,NaN,NaN,NaN,NaN,3400.0,...,34934.0,NaN,NaN,NaN,NaN,NaN,891.0,202823.0,NaN,84001320.0
2000-01-10,16070.0,20453.0,NaN,8384.0,1269818.0,NaN,NaN,NaN,NaN,71800.0,...,19667.0,NaN,NaN,NaN,NaN,NaN,2249.0,278023.0,NaN,78874940.0


In [4]:
df_px.head()

,AAC/B CN Equity,AAH CN Equity,AAR-U CN Equity,AAV CN Equity,ABX CN Equity,AC CN Equity,ACB CN Equity,ACE/A CN Equity,ACE/B CN Equity,ACM/A CN Equity,...,WTE CN Equity,WTN CN Equity,WTO CN Equity,X CN Equity,XG CN Equity,YLO CN Equity,YRI CN Equity,ZL CN Equity,ZZZ CN Equity,SPTSX Index
Index Date,,,,,,,,,,,,,,,,,,,,,
2000-01-04,12.00,12.75,NaN,6.0,25.45,NaN,NaN,NaN,NaN,13.50,...,4.9332,NaN,NaN,NaN,NaN,NaN,11.5505,19.45,NaN,8202.60
2000-01-05,12.25,12.55,NaN,6.0,25.65,NaN,NaN,NaN,NaN,14.00,...,4.9830,NaN,NaN,NaN,NaN,NaN,10.8630,18.60,NaN,8119.42
2000-01-06,12.50,12.55,NaN,6.0,25.80,NaN,NaN,NaN,NaN,13.50,...,4.9830,NaN,NaN,NaN,NaN,NaN,10.7255,18.90,NaN,8114.24
2000-01-07,12.55,12.25,NaN,6.0,25.25,NaN,NaN,NaN,NaN,13.75,...,4.9332,NaN,NaN,NaN,NaN,NaN,11.5505,19.00,NaN,8429.42
2000-01-10,13.35,13.00,NaN,5.8,25.25,NaN,NaN,NaN,NaN,13.50,...,4.9332,NaN,NaN,NaN,NaN,NaN,11.5505,19.00,NaN,8602.50


In [11]:
df_total_ret.tail()

,AAC/B CN Equity,AAH CN Equity,AAR-U CN Equity,AAV CN Equity,ABX CN Equity,AC CN Equity,ACB CN Equity,ACE/A CN Equity,ACE/B CN Equity,ACM/A CN Equity,...,WTE CN Equity,WTN CN Equity,WTO CN Equity,X CN Equity,XG CN Equity,YLO CN Equity,YRI CN Equity,ZL CN Equity,ZZZ CN Equity,SPTSX Index
Index Date,,,,,,,,,,,,,,,,,,,,,
2023-03-13,NaN,NaN,NaN,28.9449,32.3241,19.14,1.02,NaN,NaN,NaN,...,153.2987,NaN,NaN,175.6363,NaN,NaN,9.3996,NaN,28.8928,35680.7120
2023-03-14,NaN,NaN,NaN,28.8056,32.2550,19.09,1.01,NaN,NaN,NaN,...,154.6632,NaN,NaN,177.7347,NaN,NaN,9.4370,NaN,28.4189,35902.7263
2023-03-15,NaN,NaN,NaN,27.3775,33.2504,18.17,0.97,NaN,NaN,NaN,...,155.4344,NaN,NaN,179.6192,NaN,NaN,9.4370,NaN,28.0666,35330.1881
2023-03-16,NaN,NaN,NaN,27.9348,32.9877,18.61,1.04,NaN,NaN,NaN,...,160.1805,NaN,NaN,179.3118,NaN,NaN,9.3498,NaN,28.0301,35635.8153
2023-03-17,NaN,NaN,NaN,26.9595,34.4118,18.11,1.00,NaN,NaN,NaN,...,158.7567,NaN,NaN,180.3009,NaN,NaN,9.8976,NaN,27.4591,35359.9502


In [6]:
df_gics.head()

,Ticker,GICS Sector
0,AAC/B CN Equity,None
1,AAH CN Equity,None
2,AAR-U CN Equity,Real Estate
3,AAV CN Equity,Energy
4,ABX CN Equity,Materials


In [7]:
df_ref.head()

,Ticker,Weight
Index Date,,
2002-01-31,0112977D CT,0.056640
2002-01-31,0132533D CT,2.797760
2002-01-31,0206904D CT,0.705212
2002-01-31,0227217Q CT,0.140393
2002-01-31,0334249Q CT,0.501815


In [8]:
df_volume.shape, df_px.shape, df_total_ret.shape, df_gics.shape, df_ref.shape

((5810, 573), (5810, 573), (5830, 573), (572, 2), (61152, 2))

In [12]:
df_volume.isna().sum().sum(), df_px.isna().sum().sum(), df_total_ret.isna().sum().sum(), df_gics.isna().sum().sum(), df_ref.isna().sum().sum()

(1159652, 1168507, 1173026, 131, 0)

In [13]:
abs(df_px.isna().sum().sum() - df_total_ret.isna().sum().sum())

4519

In [14]:
df_gics

,Ticker,GICS Sector
0,AAC/B CN Equity,None
1,AAH CN Equity,None
2,AAR-U CN Equity,Real Estate
3,AAV CN Equity,Energy
4,ABX CN Equity,Materials
...,...,...
567,XG CN Equity,None
568,YLO CN Equity,None
569,YRI CN Equity,Materials
570,ZL CN Equity,None


In [9]:
pd.read_parquet("converted_data/filtered_data.parquet")

,AAC/B CN Equity,AAH CN Equity,AAR-U CN Equity,AAV CN Equity,ABX CN Equity,AC CN Equity,ACB CN Equity,ACE/A CN Equity,ACE/B CN Equity,ACM/A CN Equity,...,WSP CN Equity,WTE CN Equity,WTN CN Equity,WTO CN Equity,X CN Equity,XG CN Equity,YLO CN Equity,YRI CN Equity,ZL CN Equity,ZZZ CN Equity
Index Date,,,,,,,,,,,,,,,,,,,,,
2000-01-04,8750.0,9711.0,NaN,16575.0,926579.0,NaN,NaN,NaN,NaN,21396.0,...,NaN,25607.0,NaN,NaN,NaN,NaN,NaN,NaN,510922.0,NaN
2000-01-05,23285.0,11400.0,NaN,31275.0,1452213.0,NaN,NaN,NaN,NaN,6000.0,...,NaN,55081.0,NaN,NaN,NaN,NaN,NaN,NaN,343030.0,NaN
2000-01-06,29305.0,8783.0,NaN,5150.0,600273.0,NaN,NaN,NaN,NaN,38400.0,...,NaN,23580.0,NaN,NaN,NaN,NaN,NaN,NaN,518189.0,NaN
2000-01-07,13695.0,11620.0,NaN,4555.0,1020813.0,NaN,NaN,NaN,NaN,3400.0,...,NaN,34934.0,NaN,NaN,NaN,NaN,NaN,NaN,202823.0,NaN
2000-01-10,16070.0,20453.0,NaN,8384.0,1269818.0,NaN,NaN,NaN,NaN,71800.0,...,NaN,19667.0,NaN,NaN,NaN,NaN,NaN,NaN,278023.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-22,NaN,NaN,NaN,1228248.0,6959963.0,2948587.0,3467182.0,NaN,NaN,NaN,...,199407.0,162937.0,NaN,NaN,55049.0,NaN,NaN,1676407.0,NaN,160819.0
2022-12-23,NaN,NaN,NaN,898735.0,6510750.0,2199482.0,1811017.0,NaN,NaN,NaN,...,123579.0,114166.0,NaN,NaN,79987.0,NaN,NaN,2852294.0,NaN,161553.0
2022-12-28,NaN,NaN,NaN,1477605.0,10668831.0,4472163.0,4727413.0,NaN,NaN,NaN,...,184946.0,142018.0,NaN,NaN,92335.0,NaN,NaN,1810390.0,NaN,68716.0


In [10]:
weights = pd.read_parquet("converted_data/base_strategy_weights.parquet")
weights

,AAC/B CN Equity,AAH CN Equity,AAR-U CN Equity,AAV CN Equity,ABX CN Equity,AC CN Equity,ACB CN Equity,ACE/A CN Equity,ACE/B CN Equity,ACM/A CN Equity,...,WSP CN Equity,WTE CN Equity,WTN CN Equity,WTO CN Equity,X CN Equity,XG CN Equity,YLO CN Equity,YRI CN Equity,ZL CN Equity,ZZZ CN Equity
Index Date,,,,,,,,,,,,,,,,,,,,,
2000-12-29,0.000936,0.000547,NaN,0.000493,0.008669,NaN,NaN,NaN,NaN,0.000493,...,NaN,0.006631,NaN,NaN,NaN,NaN,NaN,NaN,0.005421,NaN
2001-12-31,0.000797,0.000595,NaN,0.005524,0.009994,NaN,NaN,NaN,NaN,0.000490,...,NaN,0.001198,0.000920,0.000490,NaN,NaN,NaN,NaN,0.005886,NaN
2002-12-31,0.000694,0.000899,NaN,0.006092,0.009378,NaN,NaN,NaN,NaN,0.000756,...,NaN,0.000492,0.001968,0.000492,NaN,NaN,NaN,NaN,0.005384,NaN
2003-12-31,0.000492,0.001596,NaN,0.006413,0.003651,NaN,NaN,NaN,NaN,0.000492,...,NaN,0.002580,0.001457,0.000737,NaN,NaN,NaN,NaN,0.002414,NaN
2004-12-31,0.000995,0.000724,NaN,0.008987,0.007879,NaN,NaN,NaN,NaN,0.000685,...,NaN,0.005371,0.001371,0.000607,NaN,NaN,0.001215,0.001551,0.004727,NaN
2005-12-30,0.001154,0.000679,NaN,0.009175,0.004773,NaN,NaN,0.002505,0.001494,0.001601,...,NaN,0.004009,0.002025,0.003084,NaN,NaN,0.005390,0.001242,0.001405,NaN
2006-12-29,0.001445,0.001094,NaN,0.005319,0.006793,NaN,NaN,0.002719,0.003053,0.002252,...,NaN,0.002533,0.001152,0.002862,NaN,NaN,0.005071,0.004499,0.001313,NaN
2007-12-31,0.001769,0.001209,NaN,0.003863,0.005974,0.000889,NaN,0.001178,0.000492,0.002593,...,0.000492,0.003876,0.000793,0.001864,NaN,NaN,0.004933,0.005208,0.001889,NaN
2008-12-31,NaN,0.000693,NaN,0.001114,0.007067,0.001794,NaN,0.003700,0.000489,0.000583,...,0.002223,0.001336,0.000489,NaN,NaN,NaN,0.000793,0.006759,0.001033,NaN
